In [ ]:
import pandas as pd
import numpy as np
import holoviews as hv
from holoviews import opts, dim

![](https://holoviews.org/_static/logo_horizontal_theme.png)
## ¿Qué es Holoviews?

[HoloViews](https://holoviews.org/index.html) es una librería de gráficos de alto nivel que forma parte del ecosistema [HoloViz](). Permite la especificación de gráficos y es independiente de lo que se utiliza para representarlos. Usaremos Bokeh como nuestro renderizador.

Para configurar esto, importamos HoloViews (como hv) y luego configuramos la extensión de Holoviews para que sea Bokeh usando hv.extension('bokeh') en la parte superior del cuaderno.

## ¿Cómo funciona?
Supongamos que tenemos un dataset en formato [tidy](https://es.wikipedia.org/wiki/Tidy_data), es decir, cada fila es un registro y cada variable es un atributo de cada registro. Ya está lógicamente organizado; cada fila es una observación y cada columna una variable. Tratemos de pensar, en primer término, de forma conceptual y no en términos de cómo vamos a escribir un código: ¿cómo hacemos para generar un gráfico de dispersión a partir de ese dataset en formato tidy?

- En primer lugar, obviamete, es necesario especificar qué tipo de **elemento** gráfico queremos convertir nuestro conjunto de datos.
- Luego, necesitamos **anotar** las columnas del dataset. Es decir, debemos anotar qué columna determinará la coordenada x en el diagrama de dispersión y cuál determinará la coordenada y.
- Después de haber tomado estas decisiones, es decir, qué tipo de elemento gráfico queremos producir y qué columnas dan las coordenadas x y cuáles dan las coordenadas y, el scatter fundamental está completo. Todo lo demás supone customizar cuestiones de estilo y diseño gráficos.

La idea, entonces, de HoloViews, es mediante el agregado de anotaciones mínimas al dataset (tidy, deben estar ordenados en formati tudy!) habilitar la visualización. Luego, se puede customizar la visualización, pero la idea es que ese proceso de anotación sea suficiente para describir la estrutura básica de un gráfico. En concreto, las anotaciones necesarias:

- Qué tipo de elemento de trazado está creando (por ejemplo, dispersión, caja y bigotes, mapa de calor, etc.).
- Qué columnas especifican las dimensiones de los datos, necesarios para configurar los ejes.

Una vez que realice esas anotaciones, HoloViews puede encargarse de la representación, utilizando Matplotlib, Bokeh o Plotly. Los objetos de  HoloViews son "conceptuales" e independientes de los detalles de la representación. Podemos modificar todas las propiedades graficas de la visualización pero la estructura fundamental está definida en la anotación.

## Importando HoloViews
HoloViews se importa como `hv`, como se ve en la celda en la parte superior de este notebook. Debido a que HoloViews es independiente del renderizador final, necesitamos especificar una extensión, lo cual hicimos anteriormente al ejecutar `hv.extension('bokeh')`. Nuestros gráficos ahora se renderizarán usando Bokeh.


>**Aclaración para Colab:** como van a ver a continuación por algún glitch en Colab es necesario agregar en cada celda la instrucción  `hv.extension('bokeh')`. Si no, no van a poder ver el gráfico. Esto no ocurre cuando trabajan con Jupyter notebooks locales.


## Importando los datos
Vamos a cargar un dataset con focos de incendios georreferenciados. Existen diversas fuentes para la georreferenciación de focos de incendios. Para el presente notebook, se utilizó la información basada en focos de calor producida por el FIRMS, particularmente el producto MODIS/Aqua+Terra Thermal Anomalies/Fire locations 1km FIRMS V0061 NRT (MCD14DL) en formato vectorial. Cada localización de un foco de calor representa el centro de un píxel de aproximadamente un 1km de lado, que contiene uno o más incendios o fuegos activos. Se trata de uno de los productos más básicos en los cuales se detectan potenciales incendios y otras anomalías térmicas.

Ahora para trabajar con esta fuente se unificó en una sola tabla todos los registros desde el 01-01-2018 al 31-12-2022. La fuente brinda, entre otras, información sobre

- la latitud y longitud del centroide correspondiente a cada pixel en que se registra las anomalías
- la fecha y hora en que se tomó el dato
una medida FRP (Fire Radiative Power) que mide el poder de irradiación en el foco en MegaWatts
- una métrica sobre la “confiabilidad” del dato que varía de 0 a 100.

Hay más información sobre las diferentes columnas del dataset en [este link](https://www.earthdata.nasa.gov/learn/find-data/near-real-time/firms/mcd14dl-nrt#ed-firms-attributes).


In [ ]:
fires = pd.read_csv("https://github.com/gefero/FCEyN_Viz/raw/main/data/modis_focus_unified_depto.csv").dropna() # eliminamos algunos na que andan dando vuelta


A su vez, a este dataset, le agregamos tres columnas:

- `link`: codigo de provincia y departamento
- `provincia_cna18`: etiqueta de provincia
- `depto_cna18`: etiqueta de departamento

Generamos algunas variables de fecha que vamos a usar más adelante

In [ ]:
fires['day'] = pd.DatetimeIndex(fires['acq_date']).day
fires['month'] = pd.DatetimeIndex(fires['acq_date']).month
fires['year'] = pd.DatetimeIndex(fires['acq_date']).year
fires['ym'] = pd.to_datetime(fires['acq_date']).dt.strftime('%Y-%m')
fires.head()

,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,bright_t31,frp,daynight,link,provincia_cna18,depto_cna18,day,month,year,ym
0,313.7,1.3,1.1,2019-01-01,1436,Terra,MODIS,24,292.1,10.1,D,86014.0,santiago del estero,alberdi,1,1,2019,2019-01
1,320.3,1.0,1.0,2019-01-01,1436,Terra,MODIS,54,304.1,9.0,D,86133.0,santiago del estero,pellegrini,1,1,2019,2019-01
2,315.2,1.4,1.2,2019-01-01,1436,Terra,MODIS,40,291.9,12.6,D,22007.0,chaco,almirante brown,1,1,2019,2019-01
3,312.1,1.4,1.2,2019-01-01,1436,Terra,MODIS,11,289.7,7.9,D,22007.0,chaco,almirante brown,1,1,2019,2019-01
4,320.6,1.4,1.2,2019-01-01,1437,Terra,MODIS,8,298.8,17.4,D,86119.0,santiago del estero,moreno,1,1,2019,2019-01


## Un ejemplo inicial simple: scatterplot
Hagamos un gráfico de dispersión y veamos cómo se relaciona la sintaxis con las ideas detrás de anotar conjuntos de datos.


In [ ]:
hv.extension('bokeh')
hv.Points(data=fires,
                    kdims=['bright_t31', 'frp'],
                    vdims=['satellite'])

Usamos `hv.Points` para invocar un elemento (`Element`) de visualización. Un `Element` es simplemente una forma de convertir la naturaleza tabular de los datos en una representación gráfica, en este caso, un diagrama de dispersión de puntos.

Es decir, queremos hacer un gráfico en el que cada glifo se encuentre en un gráfico bidimensional y los valores de los ejes x e y sean independientes.

Los tipos de elementos disponibles se pueden encontrar en la [galería de referencia de HoloViews](https://holoviews.org/reference/index.html).

Existe otra forma de generar un gráfico similar y es usando `hv.Scatter`. La diferencia fundamental es que `hv.Scatter` se usa para visualizar datos donde la variable y es dependiente, a diferencia de Points.

## Especificación de dimensiones
Hay dos tipos de dimensiones, dimensiones clave y dimensiones de valor, especificadas con los argumentos `kdims` y `vdims`, respectivamente. Podemos pensar en las dimensiones `kdims` y `vdims` como claves y valores de un diccionario (que, a su vez, puede tener `kdims` multidimensionales). Las `kdims` dimensiones de indexación, que indican en qué parte del gráfico residirán los datos de una fila. Las dimensiones de `vdims` dan información sobre cada punto de datos. En el diagrama de arriba, las dimensiones clave son la temperatura del pixel (`bright_t31`) y  la radiación emitida (`frp`). Esas columnas determinaron dónde se colocaron los elementos del gráfico.

Además, teníamos una dimensión de valor, especificada por `vdims`, que tiene información adicional asociada con cada punto de datos (el satélite con el que fue tomada la información). Esta información no se usó en el diagrama anterior, pero la pondremos en uso momentáneamente.

## Customizando gráficos
Un `Element` contiene información de dos aspectos básicos:

- los datos, lo más cerca posible de su forma original, para que puedan ser analizados y accedidos como mejor le parezca
- Metadatos que especifican cuáles son sus datos, lo que permite a HoloViews construir una representación visual adecuada para ellos.

Lo que los `Element` no contienen son los detalles interminables que uno podría querer modificar sobre la representación visual, como anchos de línea, colores, fuentes y espaciado.

Después de especificar un `Element`, podemos modificar las opciones de visualización y estilos usando la funcionalidad `hv.opts`. Para investigar qué opciones de estilo están disponibles para cada tipo de elemento de trazado, puede ingresar, por ejemplo

In [ ]:
hv.help(hv.Points)

Points

Online example: http://holoviews.org/reference/elements/bokeh/Points.html

-------------
Style Options
-------------

	alpha, angle, cmap, color, fill_alpha, fill_color, hover_alpha, hover_color, hover_fill_alpha, hover_fill_color, hover_line_alpha, hover_line_cap, hover_line_color, hover_line_dash, hover_line_join, hover_line_width, line_alpha, line_cap, line_color, line_dash, line_join, line_width, marker, muted_alpha, muted_color, muted_fill_alpha, muted_fill_color, muted_line_alpha, muted_line_cap, muted_line_color, muted_line_dash, muted_line_join, muted_line_width, nonselection_alpha, nonselection_color, nonselection_fill_alpha, nonselection_fill_color, nonselection_line_alpha, nonselection_line_cap, nonselection_line_color, nonselection_line_dash, nonselection_line_join, nonselection_line_width, palette, selection_alpha, selection_color, selection_fill_alpha, selection_fill_color, selection_line_alpha, selection_line_cap, selection_line_color, selection_line_dash, select

y obtenemos información detallada sobre qué opciones están disponibles para estilizar elementos `hv.Points`. Probemos un estilo diferente para el gráfico anterior usando `.opts()`.

In [ ]:
hv.extension('bokeh')
hv.Points(
    data=fires,
    kdims=['bright_t31', 'frp'],
    vdims=['satellite']).opts(
      alpha=0.7,
      color='#1f77b3',
      frame_height=400,
      frame_width=400,
      show_grid=True,
)


Pueden setearse opciones gráficas por default usando `hv.opts.defaults()`.

## Agrupación por dimensiones de valor
Nos había quedado una `vdim` en el elemento que creamos. Naturalmente, nos gustaría separar los datos por satélite. Para hacer esto, podemos hacer una operación `groupby` en el `Element`. Así es, ¡podemos hacer operaciones de agrupación en elementos gráficos!

In [ ]:
hv.extension('bokeh')

hv.Points(
    data=fires,
 kdims=['bright_t31', 'frp'],
    vdims=['satellite', 'provincia_cna18']
).groupby(
    ['satellite', 'provincia_cna18']
)

Ahora tenemos un menú desplegable a la derecha del gráfico donde podemos seleccionar el satélite que queremos. Por defecto, después de aplicar la operación `groupby`, HoloViews nos da un objeto `HoloMap`. Los valores en la columna por la que solíamos agrupar ahora se pueden seleccionar a través de una interfaz gráfica (un menú desplegable).

Para tener una idea de qué está pasando por detrás, pensemos en ese objeto `HoloMap` como si fuera un diccionario... que de hecho, lo es. Entonces, lo que está pasando es algo como lo que sigue.

Primero generamos un diccionario cuya `kdim` es el satelite y sus valores son diferentes elementos `hv.Point`

In [ ]:
dicc = {sat:hv.Points(fire,
                            kdims=['bright_t31', 'frp'],
                            vdims=['satellite'])
        for sat, fire  in fires[['bright_t31','frp','satellite']].groupby('satellite')}

Luego, lo pasamos argumento a un `hv.HoloMap`

In [ ]:
hv.extension('bokeh')
hv.HoloMap(dicc, kdims='satellite')


Volvamos al hilo incial. Podemos agrupar por satélite y colocar cada plot uno al lado del otro, creando un `layout`. Podemos usar el método `layout()` para hacer esto.

In [ ]:
hv.extension('bokeh')

hv.Points(
    data=fires,
 kdims=['bright_t31', 'frp'],
    vdims=['satellite']
).groupby(
    ['satellite']
).opts(
    frame_height=400,
    frame_width=400,
).layout(
)

Podríamos pensar en otro tipo de layout. Por ejemplo, supongamos que queremos agregar dos histogramas al costado del scatter inicial. Podemos utilizar el operador `+` para genera el layout

In [ ]:
hv.extension('bokeh')
gr = (hv.Points(
    data=fires,
    kdims=['bright_t31', 'frp'],
    vdims=['satellite']) +
    hv.Histogram(np.histogram(fires['frp'], bins=100), kdims=['frp']) +
    hv.Histogram(np.histogram(fires['bright_t31'], bins=100), kdims=['confidence'])).cols(2)

print(gr)

Puede vere que el objet `gr` es un `layout` de tres elementos o niveles.

- un hv.Point
- dos hv.Histogram

cada uno definido con sus `kdims` y `vdims` respectivas.

In [ ]:
hv.extension('bokeh')
gr

Finalmente, es posible que deseemos superponer las parcelas para cada especie que dividimos por satélite. Para eso podemos usar el operador `.overlay()`

In [ ]:
hv.extension('bokeh')

hv.Points(
    data=fires,
 kdims=['bright_t31', 'frp'],
    vdims=['satellite','provincia_cna18']
).groupby(
    ['satellite']
).opts(
    frame_height=400,
    frame_width=400,
).overlay(
)

Podemos seguir agregando funcionalidades. Podemos agregar tooltips mediante `opts()` y, de esta forma, obtener información adicional de las `vdims`.



In [ ]:
hv.extension('bokeh')

hv.Points(
    data=fires,
 kdims=['bright_t31', 'frp'],
vdims=['satellite','provincia_cna18']
).groupby(
    ['satellite']
).opts(
    frame_height=400,
    frame_width=400,
    tools=['hover'] # Agregamos los tooltips
).overlay(
)

Como ejemplo final, tomemos el dataset completo, permitamos que el año se seleccione a través de un HoloMap y, además, coloreemos por satélite para cada año.

In [ ]:
hv.extension('bokeh')

hv.Points(
    data=fires,
 kdims=['bright_t31', 'frp'],
vdims=['satellite','provincia_cna18']
).groupby(
    ['satellite', 'provincia_cna18']
).opts(
    frame_height=400,
    frame_width=400,
    tools=['hover'] # Agregamos los tooltips
).overlay('satellite') # hacemos overlay solamente sobre la variable satellite

## Trabajando con agregados
Como vimos hasta aquí estuvimos trabajando sobre los datos "crudos", es decir, si hacer ningún tipo de operación de agregado sobre los mismos. Pero, ¿qué pasaría si quisiéramos ver la evolución en el tiempo de la intensidad media de los incendios por provincia? Como ya vimos, nuestro dataset está en formato tidy lo cual nos va a resultar útil.

Primero, vamos a crear un objeto de HoloViews llamado `Dataset` que declara las `kdims` y las `vdims` con las que desea trabajar:

In [ ]:
fprov = hv.Dataset(fires[['ym', 'day', 'provincia_cna18', 'frp', 'confidence']],
                   kdims=[('provincia_cna18','Provincia'),('ym','Año-mes')],
                   vdims=[('frp', 'Intensidad'), ('confidence', 'Confianza de medición')])

Aquí hemos usado una sintaxis opcional basada en tuplas `(nombre, etiqueta)` para especificar una descripción más significativa para los `vdims`, mientras usamos las descripciones cortas originales para los dos `kdims`. Ahora pedimos a HoloViews que promedie todas las dimensiones restantes:

In [ ]:
fprov = fprov.aggregate(function=np.mean)
fprov

:Dataset   [provincia_cna18,ym]   (frp,confidence)

Vemos los `kdims` (entre corchetes) como los `vdims` (entre paréntesis) del `Dataset`. Debido a que puede contener combinaciones arbitrarias de dimensiones, un `Dataset` no se puede visualizar inmediatamente. No existe un mapeo único y claro de estas cuatro dimensiones en una página bidimensional, de ahí la representación textual que se muestra arriba.

Para que estos datos sean visibles, necesitaremos proporcionar un poco más de metadatos, seleccionando uno de los `Elements` que puede ayudar a responder las preguntas que queremos hacer sobre los datos. Quizás la representación más obvia de este conjunto de datos es una curva que muestra la incidencia de cada año, para cada provincia.

Podríamos extraer columnas individuales una por una del conjunto de datos original, pero ahora que hemos declarado información sobre las dimensiones, el enfoque más limpio es asignar las dimensiones de nuestro conjunto de datos a las dimensiones de un elemento usando `.to`.

In [ ]:
hv.extension('bokeh')

layout = (fprov.to(hv.Curve, ['ym'], 'frp') + fprov.to(hv.Curve, ['ym'], 'confidence')).cols(1)
layout.opts(
    opts.Curve(width=600, height=250,
               xrotation=45,
              framewise=True, #cada plot se normaliza de forma independiente,
               tools=['hover']))

Aquí especificamos dos elementos de `Curve` que muestran la evolución de la intensidad de los incendios y de la confianza de las mediciones realizadas (`kdim`) y los dispusimos en una columna vertical. Si bien solo especificamos solo el nombre corto para las dimensiones de valor, el plot muestra los nombres más largos ("Intesidad", "Confianza de la medición") que declaramos en el `Dataset`.

También se generó automáticamente un menú desplegable para seleccionar qué provoncia ver. Cada curva ignora las `vdim` no utilizadas, porque las medidas adicionales no se afectan entre sí, pero HoloViews tiene que hacer algo con cada dimensión clave para cada gráfico de este tipo. Si la provincia (o cualquier otra `kdim`) no se utiliza, dibuja o agrega de alguna manera, entonces HoloViews tiene que dejar que el usuario elija un valor para él, de ahí el widget de selección.

## Salvando un gráfico de HoloViews
Podemos guardar un gráfico de `HoloViews` (o diseño, `HoloMap`, etc.) usando la función `hv.save()`. La función va a determinar cómo desea guardar su objeto HoloViews según el sufijo del nombre del archivo de salida.

Como ejemplo, tomemos el gráfico anterior que ya estaba guardado en una variable (`layout`):

In [ ]:
hv.save(layout, 'layout.html')

## ¿Cómo seguir?
Hasta acá solamente vimos un `Element` de HoloViews. Bueno, más o menos, porque usamos algún `hv.Histogram` y un `hv.Curve`. Pero como se imaginarán hay muchos más elementos para trabajar. Vamos a mencionar dos más o menos habituales.

### Boxplot
Estos se generan con el elemento `hv.BoxWhisker`. Si hay múltiples `kdims`, los ejes se anidan automáticamente.

In [ ]:
hv.extension('bokeh')

hv.BoxWhisker(
    data=fires,
    kdims=['year','satellite'],
    vdims=['confidence'],
).opts(
    width=400,
    height=400
)

### Striplots
Usamos `hv.Scatter()` para generar strip plots. Usamos el argumento `jitter`.


In [ ]:
hv.extension('bokeh')
fires['year_str'] = fires['year'].astype(str)

hv.Scatter(
    data=fires,
    kdims=[('year_str', 'Año')],
    vdims=['confidence','satellite'],
).groupby(
    'satellite'
).opts(
    color='satellite',
    jitter=0.4,
    show_legend=False,
    width=400,
    height=250,
).layout()
